In [ ]:
import pandas as pd
#https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

In [ ]:
df_geral = pd.read_pickle('todas_noticias.pkl')
df_geral = df_geral.dropna().drop_duplicates()

## Bigramas

In [ ]:
from gensim.models.phrases import Phrases, Phraser #Usamos o pacote do gensim para captar os bigramas

#Prases() toma uma lista de palavras como input
sent = [row.split() for row in df_geral['noticias_limpas']]

#Construimos os bigramas relevantes
phrases = Phrases(sent, min_count=30, progress_per=10000)

#O Phraser() tem por objetivo cortar memémora do Phrases()
bigram = Phraser(phrases)

#Transforma o corpus nos bigramas detectados
sentences = bigram[sent]


In [ ]:
#bigram.save('./phrase')

# Word2Vec

In [ ]:
import multiprocessing
from gensim.models import Word2Vec

cores = multiprocessing.cpu_count() # Conta o numero de cores no note

## Instanciando o modelo

In [ ]:
w2v_model = Word2Vec(sg = 0, #Skipgram ou cbow
                     min_count=20, #ignora todas as palavras com frequência absoluta menor que essa
                     window=2, #distancia máxima da palavra atual pra palavra predita
                     vector_size=300, #Dimensão do vetor
                     sample=6e-5, #O limite para configurar quais palavras de alta frequência são aleatoriamente reduzidas
                     alpha=0.03, #learning rate inicial
                     min_alpha=0.0007, # o learning rate vai diminuindo linearmente até esse patamar
                     negative=20, #Se> 0, a amostragem negativa será usada, o int para negativo especifica quantas "palavras de ruído" devem ser eliminadas. Se definido como 0, nenhuma amostra negativa é usada
                     workers=cores-1)


## Treinando o vocabulário

In [ ]:
w2v_model.build_vocab(sentences, progress_per=10000)

## Treinando o modelo

In [ ]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(657901079, 851023770)

In [ ]:
w2v_model.wv.most_similar("lula")

[('petista', 0.7483488321304321),
 ('ex_presidente', 0.7451997995376587),
 ('luiz_inacio', 0.7196370959281921),
 ('lula_silva', 0.6805077195167542),
 ('pt', 0.654815137386322),
 ('presidiario', 0.626659631729126),
 ('moro', 0.6232565641403198),
 ('lider_petista', 0.611538290977478),
 ('haddad', 0.6115062236785889),
 ('meliante_petista', 0.6089910268783569)]

## Salvando o modelo

In [ ]:
nome_modelo = 'noticias_gerais_cbow'
w2v_model.wv.save_word2vec_format("/Users/rhenanqueiroz/Documents/GitHub/fakenews/"+nome_modelo+".txt", binary=False)

## Treinando e salvando o modelo skipgram

In [ ]:
w2v_model_sg = Word2Vec(sg = 1, #Skipgram ou cbow
                     min_count=20, #ignora todas as palavras com frequência absoluta menor que essa
                     window=3, #distancia máxima da palavra atual pra palavra predita
                     vector_size=300, #Dimensão do vetor
                     sample=6e-5, #O limite para configurar quais palavras de alta frequência são aleatoriamente reduzidas
                     alpha=0.03, #learning rate inicial
                     min_alpha=0.0007, # o learning rate vai diminuindo linearmente até esse patamar
                     negative=20,
                     workers=cores-1) #Se> 0, a amostragem negativa será usada, o int para negativo especifica quantas "palavras de ruído" devem ser eliminadas. Se definido como 0, nenhuma amostra negativa é usada
w2v_model_sg.build_vocab(sentences, progress_per=10000)
w2v_model_sg.train(sentences, total_examples=w2v_model_sg.corpus_count, epochs=30, report_delay=1)

(657915689, 851023770)

In [ ]:
w2v_model_sg.wv.most_similar("lula")

[('ex_presidente', 0.8104825615882874),
 ('luiz_inacio', 0.7799150943756104),
 ('lula_silva', 0.7678058743476868),
 ('petista', 0.7453047037124634),
 ('pt', 0.6569086909294128),
 ('lula_preso', 0.6365054845809937),
 ('moro', 0.6184068918228149),
 ('haddad', 0.6036134958267212),
 ('dilma', 0.5809354782104492),
 ('fernando_haddad', 0.5759023427963257)]

In [ ]:
nome_modelo2 = 'noticias_gerais_skipg'
w2v_model_sg.wv.save_word2vec_format("/Users/rhenanqueiroz/Documents/GitHub/fakenews/"+nome_modelo2+".txt", binary=False)

## Lendo o modelo

In [ ]:
from gensim.models import KeyedVectors
w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/Users/rhenanqueiroz/Documents/GitHub/fakenews/"+nome_modelo+".txt")

In [ ]:
from gensim.models import KeyedVectors
w2v_modelo_skipg = KeyedVectors.load_word2vec_format("/Users/rhenanqueiroz/Documents/GitHub/fakenews/"+nome_modelo2+".txt")

## Aplicação

In [ ]:
import re
import numpy as np
import nltk as nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

#Reload do bigrama treinado no word2vec
bigram_reloaded = Phraser.load('./phrase')

def limpa_texto(texto):
    ''' Função para limpar textos e formata-los'''

    # remove \n e \r
    c = re.sub(r'\n', ' ', texto)
    c = re.sub(r'\r', ' ', c)

    # remove emoction
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

    c = emoji_pattern.sub(r'', c)

    # remove os @ listados
    c = re.sub(r'@[\w./-]+', '', c)

    # remove os sites
    c = re.sub(r'http[\w./:-]+', '', c)

    # remove tag html
    c = re.sub(r'<.*?>','', c)

    # remove caracteres não alfabéticos
    c = c.replace(':', ' ')
    c = c.replace(';', ' ')
    c = c.replace('.', ' ')
    c = c.replace('/', ' ')
    c = re.sub(r'R\$', '', c)
    c = re.sub(r'\W', ' ', c)
    c = re.sub('[‘’“”…]', '', c)
    c = re.sub('\w*\d\w*', '', c)

    # remove espaços adicionais
    c = re.sub(r'\s+', ' ', c)

    # remove espacos adicionais no inicio das frases
    c = re.sub(r'^\s+', '', c)

    # remove espacos adicionais no final das frases
    c = re.sub(r'\s+$', '', c)

    #remove palavras que tenham mais de 3 letras repetidas seguidas e qq outro
    c = re.sub(r'\w+?(\w)\1{2,}(\w)+\b', '', c)

    #remove risadas
    c = re.sub(r'(h?a*ha+h[ha]*.)', '', c)

    #retira acentos
    c = c.replace("á","a")
    c = c.replace("ã","a")
    c = c.replace("õ","o")
    c = c.replace("é","e")
    c = c.replace("í","i")
    c = c.replace("ó","o")
    c = c.replace("ô","o")
    c = c.replace("ú","u")
    c = c.replace("ç","c")

    return c


def remove_stop(texto):
    '''Função para remover stopwords'''

    doc = nltk.word_tokenize(texto)
    return " ".join(token for token in doc if token not in stopwords)


def bigramas(texto, modelo_bigrama):
    '''Função para criar bigramas e retornar uma lista de tokens'''

    sent = modelo_bigrama[nltk.word_tokenize(texto)]
    return sent

def combinacao_de_vetores_por_soma(tokens, modelo):
    '''Função para somar cada token considerando que a palavra esteja num
    vetor do modelo word2vec. Caso não esteja, somente será repassado'''

    vetor_resultante = np.zeros((1,300))

    for token in tokens:
        try:
            vetor_resultante += modelo.get_vector(token)

        except KeyError:
            pass

    return vetor_resultante

def matriz_vetores(textos, modelo_word2vec, modelo_bigrama):
    ''' Função para automatizar a transformação do texto em um vetor único'''
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        texto_limpo = limpa_texto(textos.iloc[i])
        texto_limpo_semstop = remove_stop(texto_limpo)
        tokens = bigramas(texto_limpo_semstop, modelo_bigrama)
        matriz[i] = combinacao_de_vetores_por_soma(tokens, modelo_word2vec)

    return matriz


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rhenanqueiroz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Treino na base Fakebr

In [ ]:
import pandas as pd
dados = pd.read_csv('https://raw.githubusercontent.com/roneysco/Fake.br-Corpus/master/preprocessed/pre-processed.csv')
dados.head()

,index,label,preprocessed_news
0,0,fake,katia abreu diz vai colocar expulsao moldura n...
1,1,fake,ray peita bolsonaro conservador fake entrevist...
2,2,fake,reinaldo azevedo desmascarado policia federal ...
3,3,fake,relatorio assustador bndes mostra dinheiro pub...
4,4,fake,radialista americano fala sobre pt vendem ilus...


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(dados.preprocessed_news, dados.label,
                                                    test_size=0.2, random_state=28)

In [ ]:
X_testuui = X_train.sample(3).reset_index().drop(columns = 'index')
X_testuui

,preprocessed_news
0,mg governador petista nao paga funcionalismo p...
1,video memoravel ayrton senna dizendo viu deus ...
2,angelica revela defeitos marido chatinho solta...


## Exemplo

In [ ]:
X_testuui.preprocessed_news.iloc[0]

'mg governador petista nao paga funcionalismo publico ira gastar r milhoes publicidade meio denuncias riscos concretos governador fernando pimentel terminar mandato preso governo minas apagar luzes anunciou ira abrir nova licitacao informacoes bhaz verba agora sera minimo milhoes ano podendo chegar milhoes engracado desde atual gestao assumiu nao faltaram reclamacoes falta recursos assombroso ainda momento anuncia nao dinheiro pagar dia salario funcionalismo publico proximos meses governo petista coloca prioridade publicidades milionarias salario funcionarios estado nao garantido assim verba publicidade estara bem garantida gasto dessa magnitude momento tao complicado atual minimo imoral portal bhaz prometeu acompanhar perto juntamente ministerio publico todo processo estaremos olho gastanca mineira faremos questao divulgar materias portal bhaz'

In [ ]:
texto_limpo = limpa_texto(X_testuui.preprocessed_news.iloc[0])
texto_limpo

'mg governador petista nao paga funcionalismo publico ira gastar r milhoes publicidade meio denuncias riscos concretos governador fernando pimentel terminar mandato preso governo minas apagar luzes anunciou ira abrir nova licitacao informacoes bhaz verba agora sera minimo milhoes ano podendo chegar milhoes engracado desde atual gestao assumiu nao faltaram reclamacoes falta recursos assombroso ainda momento anuncia nao dinheiro pagar dia salario funcionalismo publico proximos meses governo petista coloca prioridade publicidades milionarias salario funcionarios estado nao garantido assim verba publicidade estara bem garantida gasto dessa magnitude momento tao complicado atual minimo imoral portal bhaz prometeu acompanhar perto juntamente ministerio publico todo processo estaremos olho gastanca mineira faremos questao divulgar materias portal bhaz'

In [ ]:
texto_limpo_semstop = remove_stop(texto_limpo)
texto_limpo_semstop

'mg governador petista nao paga funcionalismo publico ira gastar r milhoes publicidade meio denuncias riscos concretos governador fernando pimentel terminar mandato preso governo minas apagar luzes anunciou ira abrir nova licitacao informacoes bhaz verba agora sera minimo milhoes ano podendo chegar milhoes engracado desde atual gestao assumiu nao faltaram reclamacoes falta recursos assombroso ainda momento anuncia nao dinheiro pagar dia salario funcionalismo publico proximos meses governo petista coloca prioridade publicidades milionarias salario funcionarios estado nao garantido assim verba publicidade estara bem garantida gasto dessa magnitude momento tao complicado atual minimo imoral portal bhaz prometeu acompanhar perto juntamente ministerio publico todo processo estaremos olho gastanca mineira faremos questao divulgar materias portal bhaz'

In [ ]:
tokens = bigramas(texto_limpo_semstop, bigram_reloaded) #Usamos o bigrama já treinadado com os dados do w2v
tokens[:8]

['mg',
 'governador',
 'petista',
 'nao',
 'paga',
 'funcionalismo_publico',
 'ira',
 'gastar']

In [ ]:
vetor_resultante = matriz_vetores(X_testuui.preprocessed_news, w2v_modelo_cbow,bigram_reloaded)
vetor_resultante[:10]

array([[ 2.66494107e+01, -2.21766821e+00, -1.19723012e+01,
        -3.55663588e+01, -8.83896732e+00,  9.43865732e+00,
        -6.25194667e-01,  1.76426215e+01,  5.35734956e-01,
         1.39825954e+01,  9.53949639e-01,  8.24341543e+00,
        -2.15985280e+01,  1.11094686e+01, -2.08263880e+01,
         9.81682720e+00, -3.97188396e+00, -3.23150373e+00,
        -2.05614415e+01,  2.24751387e+00,  2.08086079e+01,
        -8.02875726e+00,  7.70033446e+00,  1.04920275e+01,
        -3.80253788e+01,  1.48237188e+01,  4.00518670e+01,
         1.47755101e+01, -2.53976892e+01,  2.65523637e+01,
         3.80916574e+01, -4.00372496e+01,  1.20396973e+01,
        -8.07135690e+00,  7.42554868e+00,  1.01521387e-01,
         2.26322415e+01,  2.73266173e+01, -2.19964301e+01,
         2.73305741e+00,  1.51471055e+01,  8.05618376e+00,
        -8.67833367e+00,  1.85732334e+01,  9.22359244e+00,
        -1.79084961e+01,  1.13389453e+01, -9.26234344e+00,
         2.74623984e+01, -3.36065056e+01,  2.47956387e+0

## Treino do classificador - Regressão logística

In [ ]:
def classificador(modelo_word2vec, modelo_bigrama, x_treino,x_teste, y_treino, y_teste):
    ''' Função que cria um classificador de reg logistica usando-se os vetores do modelo word2vec'''

    matriz_vetores_treino = matriz_vetores(x_treino, modelo_word2vec, modelo_bigrama)
    matriz_vetores_teste = matriz_vetores(x_teste, modelo_word2vec, modelo_bigrama)
    RL = LogisticRegression(max_iter = 2000)
    RL.fit(matriz_vetores_treino, y_treino)
    categorias = RL.predict(matriz_vetores_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)

    return RL

In [ ]:
RL_sg = classificador(w2v_modelo_cbow, bigram_reloaded, X_train, X_test, y_train, y_test)

              precision    recall  f1-score   support

        fake       0.94      0.96      0.95       743
        true       0.95      0.94      0.94       697

    accuracy                           0.95      1440
   macro avg       0.95      0.95      0.95      1440
weighted avg       0.95      0.95      0.95      1440



## Treino do classificador - LightGBM

In [ ]:
matriz_vetores_total_cbow = matriz_vetores(dados.preprocessed_news, w2v_modelo_cbow, bigram_reloaded)
matriz_vetores_total_skipg = matriz_vetores(dados.preprocessed_news, w2v_modelo_skipg, bigram_reloaded)
y  = dados.label.map({'true': 1, 'fake': 0})

In [ ]:
dtrain_cbow = lgb.Dataset(matriz_vetores_total_cbow, label=y)
dtrain_sg = lgb.Dataset(matriz_vetores_total_skipg, label=y)

In [ ]:
import lightgbm as lgb
def cross_val(eta=0.1,num_leaves=8,bag_frac=0.8,bag_freq=5,feat_frac=0.8):

    '''
    A function to return cross-validated LightGBM model accuracy

    Inputs
      eta: float, learning rate for model
      num_leaves: int, maximum number of leaves in tree
      bag_frac: float, random bagging fraction
      bag_freq: int, frequency of bagging
      feat_frac: float, feature fraction

    Outputs
      float, model accuracy from best iteration of cross-validated model
    '''

    # set the model parameters
    parameters = {
        'objective': 'multiclass',
        'metric': 'multi_error',
        'num_class':10,
        'learning_rate': eta,
        'num_leaves': int(num_leaves), # need to set this as int since optimisation feeds in float
        'bagging_fraction': bag_frac,
        'bagging_freq': int(bag_freq), # need to set this as int since optimisation feeds in float
        'feature_fraction': feat_frac,
        'force_col_wise':True, # suppress the warning
        'verbosity':-1
        }


    model = lgb.cv(params = parameters,
                    train_set = dtrain_cbow,
                    num_boost_round=2000,
                    stratified=False,
                    nfold = 5,
                    verbose_eval=50,
                    seed = 23,
                    early_stopping_rounds=75)


    # return accuracy rather than error
    # as the optimiser seeks to maximise rather than minimise
    return 1. - model['multi_error-mean'][-1]

In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:
# define the parameter search space
space = {
    'eta':(0.025,0.15),
    'num_leaves':(2,16),
    'bag_frac':(0.5,0.8),
    'bag_freq':(1,5),
    'feat_frac':(0.5,0.8)
    }


# set up the optimiser
optimiser = BayesianOptimization(
    f = cross_val,
    pbounds = space,
    verbose = 2,
    random_state = 0)

In [ ]:
optimiser.maximize(
    init_points=3,
    n_iter=5,
)

|   iter    |  target   | bag_frac  | bag_freq  |    eta    | feat_frac | num_le... |
-------------------------------------------------------------------------------------
[50]	cv_agg's multi_error: 0.0441667 + 0.00556596
[100]	cv_agg's multi_error: 0.0444444 + 0.00657342
|  6        |  0.9565   |  0.7375   |  3.116    |  0.09601  |  0.7777   |  2.995    |
[50]	cv_agg's multi_error: 0.0427778 + 0.00427182
[100]	cv_agg's multi_error: 0.0409722 + 0.00564169
[150]	cv_agg's multi_error: 0.0402778 + 0.0056074
[200]	cv_agg's multi_error: 0.0406944 + 0.00518932
|  7        |  0.9604   |  0.5261   |  1.081    |  0.1291   |  0.7334   |  14.18    |
[50]	cv_agg's multi_error: 0.0448611 + 0.00611111
[100]	cv_agg's multi_error: 0.0443056 + 0.0082449
[150]	cv_agg's multi_error: 0.0425 + 0.00690824
[200]	cv_agg's multi_error: 0.0426389 + 0.00637377
|  8        |  0.9586   |  0.7936   |  4.197    |  0.08268  |  0.7342   |  3.656    |
[50]	cv_agg's multi_error: 0.0440278 + 0.00735718
[100]	cv_agg's mul

In [ ]:
print(optimiser.max)

{'target': 0.9604166666666667, 'params': {'bag_frac': 0.5261387899104623, 'bag_freq': 1.0808735897613029, 'eta': 0.12907748069349226, 'feat_frac': 0.7334470252849552, 'num_leaves': 14.180170075455468}}


In [ ]:
RL_sg = classificador(w2v_modelo_skipg, bigram_reloaded, X_train, X_test, y_train, y_test)

              precision    recall  f1-score   support

        fake       0.95      0.96      0.95       743
        true       0.95      0.94      0.95       697

    accuracy                           0.95      1440
   macro avg       0.95      0.95      0.95      1440
weighted avg       0.95      0.95      0.95      1440



In [ ]:
import lightgbm as lgb
def cross_val_sg(eta=0.1,num_leaves=8,bag_frac=0.8,bag_freq=5,feat_frac=0.8):

    parameters = {
        'objective': 'multiclass',
        'metric': 'multi_error',
        'num_class':10,
        'learning_rate': eta,
        'num_leaves': int(num_leaves),
        'bagging_fraction': bag_frac,
        'bagging_freq': int(bag_freq),
        'feature_fraction': feat_frac,
        'force_col_wise':True,
        'verbosity':-1
        }

    model = lgb.cv(params = parameters,
                    train_set = dtrain_sg,
                    num_boost_round=2000,
                    stratified=False,
                    nfold = 5,
                    verbose_eval=50,
                    seed = 23,
                    early_stopping_rounds=75)


    return 1. - model['multi_error-mean'][-1]

optimiser_sg = BayesianOptimization(
    f = cross_val,
    pbounds = space,
    verbose = 2,
    random_state = 0)

In [ ]:
optimiser_sg.maximize(
    init_points=3,
    n_iter=5,
)

|   iter    |  target   | bag_frac  | bag_freq  |    eta    | feat_frac | num_le... |
-------------------------------------------------------------------------------------
[50]	cv_agg's multi_error: 0.045 + 0.006233
[100]	cv_agg's multi_error: 0.0436111 + 0.00535038
[150]	cv_agg's multi_error: 0.0413889 + 0.00505754
[200]	cv_agg's multi_error: 0.0408333 + 0.00706015
[250]	cv_agg's multi_error: 0.0409722 + 0.00745356
|  1        |  0.9603   |  0.6646   |  3.861    |  0.1003   |  0.6635   |  7.931    |
[50]	cv_agg's multi_error: 0.0443056 + 0.00670993
|  2        |  0.9569   |  0.6938   |  2.75     |  0.1365   |  0.7891   |  7.368    |
[50]	cv_agg's multi_error: 0.0441667 + 0.00556596
[100]	cv_agg's multi_error: 0.0444444 + 0.00657342
|  3        |  0.9565   |  0.7375   |  3.116    |  0.09601  |  0.7777   |  2.995    |
[50]	cv_agg's multi_error: 0.0444444 + 0.00734931
[100]	cv_agg's multi_error: 0.0436111 + 0.00584984
[150]	cv_agg's multi_error: 0.0419444 + 0.00686904
[200]	cv_agg's mult

In [ ]:
print(optimiser_sg.max)

{'target': 0.9602777777777778, 'params': {'bag_frac': 0.6646440511781975, 'bag_freq': 3.860757465489678, 'eta': 0.10034542200895549, 'feat_frac': 0.6634649548990691, 'num_leaves': 7.931167190744666}}


## Modelo final

In [ ]:
parameters = {
        'objective': 'multiclass',
        'metric': 'multi_error',
        'num_class':10,
        'learning_rate': 0.12907748069349226,
        'num_leaves': int(14.180170075455468),
        'bagging_fraction': 0.5261387899104623,
        'bagging_freq': int(1.0808735897613029),
        'feature_fraction': 0.7334470252849552,
        'force_col_wise':True,
        'verbosity':-1
        }
modelo_final = lgb.cv(params = parameters,
                    train_set = dtrain_cbow,
                    num_boost_round=2000,
                    stratified=False,
                    nfold = 5,
                    verbose_eval=50,
                    seed = 23,
                    early_stopping_rounds=75)

[50]	cv_agg's multi_error: 0.0427778 + 0.00427182
[100]	cv_agg's multi_error: 0.0409722 + 0.00564169
[150]	cv_agg's multi_error: 0.0402778 + 0.0056074
[200]	cv_agg's multi_error: 0.0406944 + 0.00518932


In [ ]:
# salvar o modelo
import pickle
with open('modelo_lgbm_cbow.pkl', 'wb') as file:
    pickle.dump(modelo_final, file)